<a href="https://colab.research.google.com/github/teerasitk/thaicomRemoteSensing/blob/main/WorkShopImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work with Google Earth Engine

 ### 1. Load library and initialize Google Earth Engine

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()
import numpy as np
import matplotlib.pyplot as plt 

### 2. Build the display function for land cover map and image

In [ ]:
import folium
def showLC(image, lc_label, aoi, num_classes, palette=None, zoom=15):
  xc, yc =  aoi.centroid().getInfo()['coordinates']
  aoi_map = folium.Map(location=[yc, xc], zoom_start=zoom)
  basemaps = {'Google Maps': folium.TileLayer(
    tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',  #google map service site
    attr = 'Google',  
    name = 'Google Maps', 
    overlay = True,
    control=True)} # tell folium that the base map is the google map. 
  basemaps['Google Maps'].add_to(aoi_map) # add google earth data into ku_map
  im_clib = image.clip(aoi)
  if  palette is None:
     palette = ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4',
                'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
  viz_params = {'bands':[lc_label], 
                'min': 0, 
                'palette': palette,
                'max':num_classes} 
  map_obj = im_clib.getMapId(viz_params) # convert image into map object

  folium.TileLayer(
      tiles=map_obj ['tile_fetcher'].url_format, #item where the image is linked to
      overlay=True,
      attr='Original Image',
      name=f'False Color Composite',
    ).add_to(aoi_map)
  return aoi_map

def showImageOnMap(image, bands, aoi, min_val, max_val, zoom=15):
  xc, yc =  aoi.centroid().getInfo()['coordinates']
  aoi_map = folium.Map(location=[yc, xc], zoom_start=zoom)
  basemaps = {'Google Maps': folium.TileLayer(
    tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',  #google map service site
    attr = 'Google',  
    name = 'Google Maps', 
    overlay = True,
    control=True)} # tell folium that the base map is the google map. 
  basemaps['Google Maps'].add_to(aoi_map) # add google earth data into ku_map
  im_clib = image.clip(aoi)
  viz_params = {'bands':bands, 'min': min_val, 
                'max': max_val} 
  map_obj = im_clib.getMapId(viz_params) # convert image into map object

  folium.TileLayer(
      tiles=map_obj ['tile_fetcher'].url_format, #item where the image is linked to
      overlay=True,
      attr='Original Image',
      name=f'False Color Composite',
    ).add_to(aoi_map)
  return aoi_map
def getNewBandNames(prefix, num):
  return [f"{prefix}{k}" for k in range(num)]
def normalizeAllBand(image, aoi):
  img_band_names = image.bandNames().getInfo()
  imout = None 
  for band in img_band_names:
    imb = image.select(band)
    min_max = imb.reduceRegion(ee.Reducer.percentile([2,98]),
                               geometry=aoi)#.getIngo()
    print(min_max.getInfo())
    imb = imb.unitScale(min_max.get(band+'_p2'), min_max.get(band+'_p98'))
    if imout is None:
      imout = imb
    else:
      imout = imout.addBands(imb)
  return imout

### 3. Set the area of interest (AOI)

In [ ]:
aoi = ee.Geometry.Polygon([ [ 99.776796700071003, 17.064532089073847 ],
                           [ 99.821638573909041, 17.079199991731151 ],
                           [ 99.870671277077747, 17.06620842080611 ],
                           [ 99.86710907214669, 16.987211287923198 ],
                           [ 99.774282202472605, 16.983858624458673 ], 
                           [ 99.776796700071003, 17.064532089073847 ] ])

### 4. Load a single date Image

In [ ]:
from datetime import datetime 
img = ee.ImageCollection('COPERNICUS/S2_SR')
img = img.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)).filterDate('2019-11-01','2019-12-01')
img = img.select("B.*")
img = img.filterBounds(aoi).first()
date = img.getInfo()['properties']["system:time_start"]
date = date_time_obj = datetime.fromtimestamp(date/1000.0) 
datetxt = date.strftime('%Y-%m-%d')
print(f"Image was captured on {datetxt}")
img = img.multiply(0.0001)

### 5. Display an image

In [ ]:
im_map = showImageOnMap(img, ["B8","B4", "B3"],aoi, min_val=0, max_val=0.5, zoom=14)
im_map

### 6. Load Ground data from ESA WorldCover 10m v100 dataset

In [ ]:
dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
date = dataset .getInfo()['properties']["system:time_start"]
date = date_time_obj = datetime.fromtimestamp(date/1000.0) 
datetxt = date.strftime('%Y-%m-%d')
print(f"Dataset was creted on {datetxt}")
remap_values = ee.List([0, 1,1, 2, 3, 4, 4, 5, 4, 4, 4])
label = "landcover"
legend = ['Trees', 'Shrubland-Grassland', 'Cropland', 'Built-up', 'Barren / sparse vegetation', 'Open water']
palette = ['006400' ,'00FA00', 'ffff4c', 'fa0000', 'f000f0', '0000fa']
class_values = ee.List([10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100])
dataset = dataset.remap(class_values, remap_values).rename(label).toByte()

### 7. Display the ground data where
- Dark Green: Trees
- Light Green: Shrubland and Grassland
- Yellow: Cropland
- Red: Built-up
- Purple: Barren / sparse vegetation
- Blue: Open Water


In [ ]:
lc_map = showLC(dataset, label, aoi, 5, zoom=14, palette=palette)
lc_map

### 8. Take 100 samples per class

In [ ]:
sample = img.addBands(dataset).stratifiedSample(numPoints=100, 
                                                classBand=label,
                                                region=aoi,
                                                scale=10,
                                                geometries=True)

### 9. Check how many samples that we can actually collected

In [ ]:
for cl, name in enumerate(legend):
  num = sample.filter(f'landcover=={cl}').aggregate_count("landcover").getInfo()
  print(f"Sample Class {name}: {num}")

### 10. Make 70% Train and 30% Test Samples

In [ ]:
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.7')
validationSample = sample.filter('random > 0.7')

### 11. Build Support Vector Machine with RBF kernel function with $\gamma=5$ 

$\phi(u,v)= e^{-\gamma|u-v|^2}$

and penalty $C=100$

In [ ]:
trainedClassifier = ee.Classifier.libsvm(kernelType="RBF",gamma=5,cost=100)
trainedClassifier = trainedClassifier.train(features=trainingSample, 
                                            classProperty=label,
                                            inputProperties=img.bandNames())

### 12. Check on the description of the classifier

In [ ]:
trainedClassifier.explain().getInfo()

### 13. Obtain confusion matrix on both train and validation samples

In [ ]:
train_conf_mat = np.array(trainedClassifier.confusionMatrix().getInfo())
validationResult = validationSample.classify(trainedClassifier)
validation_conf_mat = np.array(validationResult.errorMatrix(label, 'classification').getInfo())

Confusion matrix for Train Samples

In [ ]:
print(train_conf_mat)

[[52  7  3  2  3  0]
 [ 6 51  7  0  6  0]
 [ 6  1 56  2  7  0]
 [ 5  0  0 65  8  0]
 [ 7  7  8  7 44  0]
 [ 0  0  0  0  2 68]]


Confusion matrix for validation Samples

In [ ]:
print(validation_conf_mat)

#### 12.1 Compute OA, Kappa, and Producers and Users' accuracies on Train

In [ ]:
def evaluteReport(conf_mat, legend=None):
  diag_conf_mat = np.diag(conf_mat)
  N = conf_mat.sum()
  N_ref = conf_mat.sum(0)
  N_map  = conf_mat.sum(1)
  pc = diag_conf_mat.sum()/ N
  ua = diag_conf_mat/ N_map
  pa = diag_conf_mat/ N_ref
  pe = (N_map * N_ref).sum() / (N**2)
  kappa = (pc - pe)/(1-pe)
  print(f"OA: {pc:0.3f} with Kappa: {kappa:0.3f}")
  if legend is None:
    legend = np.arange(conf_mat.shape[0])
  for k, name in enumerate(legend):
    print(f"PA[{name}]: {pa[k]:0.3f}, UA[{name}]: {ua[k]:0.3f}")
  return pc, kappa, ua, pa

Evaluate on train samples

In [ ]:
_ = evaluteReport(train_conf_mat, legend)

#### 12.2 Compute OA, Kappa, and Producers and Users' accuracies on Validation Samples

In [ ]:
_ = evaluteReport(validation_conf_mat, legend)

By comparing the results from train and validation samples, we observed that accuracies on both samples are quite similar excepts on Barren / sparse vegetatio Class. 

However, the validation accuracies seems to be smaller than those on the train samples.

### 13. Display the resulting land cover map

In [ ]:
imgClassified = img.classify(trainedClassifier);
lc_map = showLC(imgClassified, "classification", aoi, 5, zoom=14, palette=palette)
lc_map

### 14. Let evaluate on the test samples

In [ ]:
test_sample = img.addBands(dataset).sample(numPixels=1000,    # get another 1000 test samples
                                           region=aoi,
                                           scale=10,
                                           geometries=True)
for cl, name in enumerate(legend):
  num = test_sample.filter(f'landcover=={cl}').aggregate_count("landcover").getInfo()
  print(f"Sample Class {name}: {num}") # how many samples per class
test_result = test_sample.classify(trainedClassifier)
test_conf_mat = np.array(test_result.errorMatrix(label, 'classification').getInfo())
_ = evaluteReport(test_conf_mat, legend)

From the test and validation results, we found that the accuracies on the test samples are low.  

# Assignment 1
Let us try to improve the accuracies by increasing the number of samples to 1,000 per class.

Report on Train, Validation and Test Accuracies

In [ ]:
# You code is here

## Result for 1,000 samples:__________. 
What is your obsevation after increasing the number of samples

# Assignment 2

Maybe the poor accuracies on the test samples are due to the fact that we employed the inferior classifier. In this examples, we will use a more sophisticated classifier, namely, the Random Forest. To initialize the Random Forest we will write
stateOfTheArtClassifier = ee.Classifier.smileRandomForest(100, maxNodes=1024). 
Try the new classifier with 1000 samples

In [ ]:
# You code is here

Report yor results on train, validation, and test samples

# Assignment 3

From the results on Assignment 2, you will see that there are big gaps between train and validation accuracies. This phenomenal is called **Overfitting** problem. The remedy to this problem can be one of combination of these strategies.

1. Increse the number of samples
2. Reduce the number of features
3. Find better feature vectors. 

In this assignment, we will first try to reduce the number of features
Here, we will use only "B2", "B3", "B4", and "B8"

In [ ]:
# You code is here

From the results, it is clear that the gap between train and validation accuracy reduces when the number of feature reduces. However, both train, validatin, and test accuracies also decrease.

# Assignment 4

Let us try to incorporate images from different dates, since crop and tree should have different temporal profile. We will use data from every one. One per month. Here, we will use all spectral bands, and increase the number of training samples to be 10,000 samples per class.

In [ ]:
# code to stack all images into a composite one
img = ee.ImageCollection('COPERNICUS/S2_SR')
img = img.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)).filterBounds(aoi)
imstack = None 
for month in range(1,13):
  str_m = f"{month:02d}"
  date0 = ee.Date(f'2019-{str_m}-01')
  date1 = date0.advance(1, "month")
  img_month = img.filterDate(date0,date1)  
  img_month = img_month.select("B.*")#.select(["B2", "B3", "B4", "B8"])#.select("B.*")#.select(["B2", "B3", "B4", "B8"])
  if img_month.size().getInfo() > 0:
    print(f"We have image on Month {month}")
    img_month = img_month.first()
    if imstack is None:
      imstack = img_month 
    else:
      imstack = imstack.addBands(img_month )

imstack = imstack.multiply(0.0001)
print(imstack.bandNames().getInfo())

Take 10,000 samples per classe

In [ ]:
sample = imstack.addBands(dataset).stratifiedSample(numPoints=10000, 
                                                    classBand=label,
                                                    region=aoi,
                                                    scale=10,
                                                    geometries=True)

 70% train 30%Test

In [ ]:
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.7')
validationSample = sample.filter('random > 0.7')

Evaluate train, validate and test data

In [ ]:
# your code here


# Work with high resolution Image

In this exercise, we will integrate the unsupervised and supervised classification for LC on high resolution image. 

### 1. Upload top_mosaic_09cm_area17.tif and gt_top_mosaic_09cm_area17.tif

### 2. Use Gdal library to read both image and ground data

In [ ]:
from osgeo import gdal 
import numpy as np
import matplotlib.pyplot as plt
im = gdal.Open("/content/top_mosaic_09cm_area17.tif")
data = im.ReadAsArray().transpose([1,2 ,0])
gt = gdal.Open("/content/gt_top_mosaic_09cm_area17.tif")
gt_data = gt.ReadAsArray().transpose([1, 2, 0])

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.imshow(gt_data)
plt.subplot(1,2,2)
plt.imshow(data)

We observe that the class of "car" has very limited number of pixels where grass, tress and builds cover the large area.

### 3. Change color in ground data into class lables.

In [ ]:
np.unique(gt_data.reshape(-1,3), axis=0)

In [ ]:
color_dict = {'building':([0,0,255], 0),
              'vegetation':([0,255,0], 1),
              'grass':([0, 255,255], 2),
              'car':([255,255,0],3),
              'pavement':([255,255,255],4)}


Convert color ground data into gray-scale image

In [ ]:
n_rows, n_cols,_ = gt_data.shape
gt_gray = np.zeros((n_rows, n_cols), "uint8")
color_list = []
for k, (name, info) in enumerate(color_dict.items()):
  value, label = info
  indx = None
  color_list.append(value) 
  for b in range(3):
    if indx is None:
      indx = (gt_data[:, :, b]==value[b])
    else:
      indx &= (gt_data[:, :, b]==value[b])
  gt_gray[indx] = label
color_list = np.array(color_list)
plt.figure(figsize=(15,10))
plt.imshow(color_list[gt_gray])

### 4. take stratified random samples

In [ ]:
def stratifiedRandomSampling(num_samples, image, ground_data):
  x = None
  y = None
  num_classes = ground_data.max() + 1
  if image.ndim == 3:
    n_rows, n_cols,n_bands = image.shape
    im1d = image.reshape((-1, n_bands))
  else:
    im1d = image
  gt1d = ground_data.flatten()
  for cls in range(num_classes):
    idx = np.nonzero(gt1d==cls)[0]
    id_rand = np.random.permutation(idx)[:num_samples]
    if x is None:
      x = im1d[id_rand,:]
      y = np.zeros((len(id_rand),), 'uint8')+ cls
    else:
      xp = im1d[id_rand,:]
      yp = np.zeros((len(id_rand),), 'uint8') + cls
      x = np.concatenate((x, xp), axis=0)
      y = np.concatenate((y, yp))
  idx = np.random.permutation(len(y))
  print(x.shape, y.shape)
  x = x[idx,:]
  y = y[idx]
  return x,y  

#### obtain 500 samples per class

In [ ]:
x,y = stratifiedRandomSampling(num_samples=500, image=data, ground_data=gt_gray)

(2500, 3) (2500,)


### 5. make train and test sample using sklearn library

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

### 6. Perform pixel-wise classificaiton using Random Forest classifier with 200 trees with maximum depth of 5.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=0)
clf.fit(x_train, y_train)
print(f"Train Accuracy: {clf.score(x_train, y_train):0.3f}")
print(f"Test Accuracy: {clf.score(x_test, y_test):0.3f}")

From the accuracy result, the classifier is overfitted!!!

### 7. perform land cover mapping on the entire scene

In [ ]:
data1d = data.reshape(-1,3)
lc_map = clf.predict(data1d)
lc_map = lc_map.reshape(n_rows, n_cols)

### 8. Display the result

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(color_list[lc_map])

### 9. Evaluate the peformance

In [ ]:
legend_high_res = list(color_dict.keys())

In [ ]:
from sklearn.metrics import confusion_matrix
conf_train = confusion_matrix(y_train, clf.predict(x_train))
conf_valid = confusion_matrix(y_test, clf.predict(x_test))
conf_whole =  confusion_matrix(gt_gray.flatten(), lc_map.flatten())
legend_high_res = list(color_dict.keys())
print("TRAIN SAMPLES")
_ = evaluteReport(conf_train, legend_high_res)
print("TEST SAMPLES")
_ = evaluteReport(conf_valid, legend_high_res)
print("WHOLE IMAGE")
_ = evaluteReport(conf_whole, legend_high_res)

### 10. Let us integrate the clustering algorithm with the supervised classification by performning superpixel clustering using SEEDS algorithm for $N=1000$ super-pixels.

In [ ]:
import cv2 
num_pixels = data.shape[0]  * data.shape[1]
num_sub_pixels = 1000
seeds = cv2.ximgproc.createSuperpixelSEEDS(data.shape[1], 
                                           data.shape[0], 
                                           data.shape[2],
                                           num_superpixels=num_sub_pixels,
                                           num_levels=5)

seeds.iterate(data, 500)
mask = seeds.getLabelContourMask(thick_line=True)
mask = mask[:,:,np.newaxis]
dat2 = (mask==255)*np.array([[0,255,255]]) + (mask==0)*data
plt.figure(figsize=(15,10))
plt.imshow(dat2)
plt.title(f"K={num_sub_pixels}")

### 11. Obtain superpixel lables

In [ ]:
spxl_im = seeds.getLabels()
print(np.unique(spxl_im))

### 12. Convert superpixel into feature vector. Here, we extract band-wise mean and standard deviation. For one superpixel, we have 6 values (3 for means and 3 for standard deviation

In [ ]:
from scipy import stats
spxl_im1d = spxl_im.flatten()
data1d = data.reshape(-1,3)
gt_gray_1d = gt_gray.flatten()
features =[]
spxl_labels = []
for val in np.unique(spxl_im1d):
  patch = data1d[spxl_im1d==val,:]
  lb_patch = gt_gray_1d[spxl_im1d==val]
  mode = stats.mode(lb_patch)
  spxl_labels.append(mode[0][0])
  #ft = [np.concatenate((patch.mean(0), patch.std(0)))]
  ft= patch.mean(0)
  features.append(ft)
features = np.array(features).reshape(-1,3)
spxl_labels = np.array(spxl_labels)

### 13. Due to limited number of super pixels, we use 20% for train and 80% for test

In [ ]:
xsp,ysp = stratifiedRandomSampling(num_samples=20, image=features, ground_data=spxl_labels)

### 14. Here, we employed the k-nearest neighbors with neighbor size of 7.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnmodel = KNeighborsClassifier(n_neighbors=7)
knnmodel.fit(xsp, ysp)

### 15. Perform classification on superpixels and put the labels into the image

In [ ]:
sp_map = knnmodel.predict(features)
lc_map2 = np.zeros((n_rows, n_cols), 'uint8')
for k,val in enumerate(np.unique(spxl_im1d)):
  lc_map2[spxl_im==val] = sp_map[k]
plt.figure(figsize=(15,10))
plt.imshow(color_list[lc_map2])

### 16. Compare between different approaches

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(color_list[gt_gray])
plt.title("Ground Data")
plt.subplot(2,2,2)
plt.imshow(color_list[lc_map])
plt.title("Pixel-wise classification")
plt.subplot(2,2,3)
plt.imshow(color_list[lc_map2])
plt.title("Superpixel Classification")

# Assignment 5
Compare the accuracies (Overall, User's, Producers' accuracies) of superpixel classification againts pixelwise approach. Which one is better?

In [ ]:
# Your code

# Assignment 6
Repeat the superpixel classification with $N=10,000$ superpixels with 250 training pixels per class.

Between $N=1,000$ and $N=10,000$ super-pixels which one perform better?

In [ ]:
# your code

# Assignment 7
Repeat the superpixel classification with $N=10,000$ superpixels with 250 training pixels per class with Random Forest Classifier with the same parameter with pixelwise.

Compare between K-nearest neighbor and Random Forest. Which one has higer accuracy?

In [ ]:
# your code